Importo las librerías necesarias

In [134]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
import pyarrow
import pickle

Traigo el dataset de entrenamiento

In [135]:
df_train = pd.read_parquet('train.parquet')

Genero la columna categoy_price, que va a ser mi variable dependiente.

In [136]:
def categorise(row):  # Le asigno valor 1 a todos los valores de la columna 'price' que estén entre 0 y 999 dólares, y el valor 0 a los que superen dicho monto.
    if row['price'] >= 0 and row['price'] <= 999:
        return 1
    else: return 0
df_train['category_price'] = df_train.apply(lambda row: categorise(row), axis=1) # La función lambda agrega las categorías creadas por la función "categorize" a la nueva columna.

In [137]:
df_train.head()

,id,url,region,region_url,price,type,sqfeet,beds,baths,cats_allowed,...,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state,category_price
0,7048013474,https://boise.craigslist.org/apa/d/very-nice-b...,boise,https://boise.craigslist.org,1350,house,1200,2,2.0,1,...,0,0,w/d in unit,detached garage,https://images.craigslist.org/00B0B_cPiJMEheZe...,Super cute row house in the Boise bench area. ...,43.5851,-116.2250,id,0
1,7043931179,https://cosprings.craigslist.org/apa/d/colorad...,colorado springs,https://cosprings.craigslist.org,1115,apartment,694,1,1.0,1,...,0,0,w/d in unit,carport,https://images.craigslist.org/00R0R_5XAoSKvfrz...,Windtree Apartment Homes currently has a spaci...,38.9137,-104.7800,co,0
2,7048254516,https://norfolk.craigslist.org/apa/d/virginia-...,norfolk / hampton roads,https://norfolk.craigslist.org,1129,apartment,900,2,2.0,0,...,0,0,w/d hookups,off-street parking,https://images.craigslist.org/00f0f_3ZbTFrsHpZ...,Call Today! show contact info Indian Lakes ...,36.7922,-76.1643,va,0
3,7041032577,https://phoenix.craigslist.org/nph/apa/d/phoen...,phoenix,https://phoenix.craigslist.org,1580,house,1469,3,2.0,1,...,0,0,w/d in unit,None,https://images.craigslist.org/00f0f_aXV5Dkd5qk...,Cody Anderson Two Brothers Realty License #: S...,33.5623,-112.0560,az,0
4,7048588701,https://nashville.craigslist.org/apa/d/antioch...,nashville,https://nashville.craigslist.org,995,apartment,700,1,1.0,1,...,0,0,w/d in unit,carport,https://images.craigslist.org/00606_gYOGKClOHv...,To schedule a tour We now book our tour appoin...,36.0595,-86.6592,tn,1


Genero dummies de la columna type para usarla en mi modelo

In [138]:
df_dummies = pd.get_dummies(df_train, columns=['type'])

Quito las columnas que no están en el set de testeo

In [139]:
df_dummies.drop(['type_land', 'type_assisted living'], axis=1, inplace=True)

In [140]:
feature_cols = df_dummies[['sqfeet','beds', 'baths', 'comes_furnished', 'type_apartment',
'type_condo', 'type_cottage/cabin', 'type_duplex', 'type_flat',
'type_house', 'type_in-law','type_loft',
'type_manufactured', 'type_townhouse' ]]
feature_cols.head()

,sqfeet,beds,baths,comes_furnished,type_apartment,type_condo,type_cottage/cabin,type_duplex,type_flat,type_house,type_in-law,type_loft,type_manufactured,type_townhouse
0,1200,2,2.0,0,0,0,0,0,0,1,0,0,0,0
1,694,1,1.0,0,1,0,0,0,0,0,0,0,0,0
2,900,2,2.0,0,1,0,0,0,0,0,0,0,0,0
3,1469,3,2.0,0,0,0,0,0,0,1,0,0,0,0
4,700,1,1.0,0,1,0,0,0,0,0,0,0,0,0


Separo las variables independientes de la variable independiente

In [141]:
X = feature_cols
y = df_dummies.category_price

Divido set de entrenamiento y testeo

In [142]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

Inicio la función del árbol de decisión

In [143]:
clf = DecisionTreeClassifier()

Entreno el modelo

In [144]:
clf = clf.fit(X_train,y_train)

Saco la predcción

In [145]:
y_pred = clf.predict(X_test)

Calculo la precisión del modelo

In [146]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7418706226429616


Intento mejorar la precisión

In [147]:
clf = DecisionTreeClassifier(max_depth=5)

Vuelvo a entrenar el modelo

In [148]:
clf = clf.fit(X_train,y_train)

Lo guardo en un archivo pickle

In [149]:
pickle.dump(clf, open('tree.pkl', 'wb'))
pickled_clf = pickle.load(open('tree.pkl', 'rb'))

Hago la predicción nuevamente

In [150]:
y_pred = pickled_clf.predict(X_test)


Calculo la precisión

In [151]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6668109751404603


Traigo el set de testeo

In [152]:
df_test = pd.read_parquet('test.parquet')

Quito las columnas que no voy a utilizar

In [153]:
test_dummies = df_test.drop(['id', 'url', 'region_url', 'image_url', 'description', 'lat', 'long','laundry_options','cats_allowed', 'dogs_allowed', 'parking_options', 'smoking_allowed', 'wheelchair_access', 'electric_vehicle_charge', 'region', 'state'],axis=1, inplace=True)

Genero dummies de la columna 'type'

In [154]:
test_dummies = pd.get_dummies(df_test, columns=['type'])

In [155]:
test_dummies.columns

Index(['sqfeet', 'beds', 'baths', 'comes_furnished', 'type_apartment',
       'type_condo', 'type_cottage/cabin', 'type_duplex', 'type_flat',
       'type_house', 'type_in-law', 'type_loft', 'type_manufactured',
       'type_townhouse'],
      dtype='object')

Inicio la función de árbol de decisión

In [156]:
clf = DecisionTreeClassifier(max_depth=3)

Realizo la predicción 

In [157]:
pred = pickled_clf.predict(test_dummies)
pred

array([0, 0, 0, ..., 0, 0, 1])

Calculo accuracy

In [158]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6668109751404603


In [159]:
Monicav84 = pd.DataFrame(data=pred, columns=['pred']).set_index('pred')
Monicav84

""
pred
0
0
0
0
1
...
0
0
0


In [160]:
Monicav84.to_csv('Monicav84.csv')